In [ ]:
# @title Inspect the schema of bigquery-public-data.thelook_ecommerce.order_items
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('bigquery-public-data.thelook_ecommerce.order_items')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Setup bigquery client and formatting
from google.cloud import bigquery
from google.colab import data_table

project = 'airflow-demo-437509' # Project ID inserted based on the query results selected to explore
client = bigquery.Client(project=project)
data_table.enable_dataframe_formatter()

In [ ]:
# @title Executes the query

sql = '''# prompt: For Shipped Orders, find average, min, max, lead time in hour before Shipped. Check status need to be 'Shipped' and lead time are higher than 0

SELECT
  AVG(EXTRACT(HOUR
    FROM (shipped_at - created_at))) AS avg_lead_time_hours,
  MIN(EXTRACT(HOUR
    FROM (shipped_at - created_at))) AS min_lead_time_hours,
  MAX(EXTRACT(HOUR
    FROM (shipped_at - created_at))) AS max_lead_time_hours
FROM
  `bigquery-public-data.thelook_ecommerce.order_items`
WHERE
  status = 'Shipped'
  AND EXTRACT(HOUR
  FROM (shipped_at - created_at)) > 0;'''
query = client.query(sql)
query
# @title Render the query results

job = client.get_job(query.job_id)
df = job.to_dataframe()
df


In [ ]:
# @title Executes the query

sql1 = '''# prompt: For each month, find total_revenue, total_items, total_purchasers, total_orders where the order not Cancelled or Returned. Please sort the result with month. Also noted that total revenue retail price multiply with num_of_item

SELECT
  EXTRACT(MONTH
  FROM
    order_items.created_at) AS month,
  SUM(order_items.sale_price * order_items.id) AS total_revenue,
  COUNT(order_items.id) AS total_items,
  COUNT(DISTINCT order_items.user_id) AS total_purchasers,
  COUNT(DISTINCT order_items.order_id) AS total_orders
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` AS order_items
WHERE
  order_items.status != 'Cancelled'
  AND order_items.status != 'Returned'
GROUP BY
  1
ORDER BY
  1;'''
query1 = client.query(sql1)
query1
# @title Render the query results

job1 = client.get_job(query1.job_id)
df1 = job1.to_dataframe()
df1


In [ ]:
# @title Line

import altair as alt

chart1 = alt.Chart(data=df1, mark={
  "type": "line",
  "interpolate": "monotone",
  "tooltip": True
}).encode(
  x={
  "field": "month",
  "type": "quantitative",
  "title": "Month",
  "axis": {
    "labelOverlap": True
  }
},
  y={
  "field": "total_revenue",
  "type": "quantitative",
  "title": "Total Revenue",
  "axis": {
    "labelOverlap": True
  }
},


)
chart1


In [ ]:
# @title Inspect the schema of bigquery-public-data.thelook_ecommerce.orders
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('bigquery-public-data.thelook_ecommerce.orders')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Executes the query

sql2 = '''# prompt: For all order, find ratio of each order status per total order

SELECT
  orders.status,
  COUNT(orders.order_id) / (
  SELECT
    COUNT(*)
  FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS orders ) AS ratio
FROM
  `bigquery-public-data.thelook_ecommerce.orders` AS orders
GROUP BY
  1;'''
query2 = client.query(sql2)
query2
# @title Render the query results

job2 = client.get_job(query2.job_id)
df2 = job2.to_dataframe()
df2


In [ ]:
# @title stacked Bar chart

import altair as alt

chart2 = alt.Chart(data=df2, mark={
  "type": "bar",
  "tooltip": True
}).encode(
  x={
  "field": "status",
  "type": "nominal",
  "title": "Status",
  "axis": {
    "labelOverlap": True
  }
},
  y={
  "field": "ratio",
  "type": "quantitative",
  "title": "Ratio",
  "axis": {
    "labelOverlap": True
  }
},


)
chart2


In [ ]:
# @title Executes the query

sql3 = '''# prompt: Find Cancelled rate based on gender

SELECT
  orders.gender,
  SUM(CASE
      WHEN orders.status = 'Cancelled' THEN 1
      ELSE 0
  END
    ) / COUNT(orders.order_id) AS cancelled_rate
FROM
  `bigquery-public-data.thelook_ecommerce.orders` AS orders
GROUP BY
  1;'''
query3 = client.query(sql3)
query3
# @title Render the query results

job3 = client.get_job(query3.job_id)
df3 = job3.to_dataframe()
df3
